In [59]:
using LinearAlgebra,SparseArrays
using DataFrames
using JuMP, Ipopt, Mosek,MosekTools,MathOptInterface

In [9]:
BranchData = DataFrame(From = [ 1, 1, 2, 3, 3, 4], To = [ 2, 3, 4, 4, 5, 5], 
    R = [ 0.0, 0.023, 0.006, 0.020, 0.0, 0.0], X = [ 0.3, 0.145, 0.032, 0.260, 0.320, 0.500], 
    Gsh = [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Bsh = [ 0.0, 0.040, 0.010, 0.0, 0.0, 0.0], 
    T = [ 1.0, 1.0, 1.0, 1.0, 0.98,1], Tl = [1.0, 1.0, 1.0, 1.0, 0.95, 1.0], Tu = [1.0, 1.0, 1.0, 1.0, 1.05, 1.0],
    ϕ = [ 0.0, 0.0, 0.0, -3.0, 0.0, 0.0], ϕl = [ 0.0, 0.0, 0.0, -30.0, 0.0, 0.0], ϕu = [ 0.0, 0.0, 0.0, 30.0, 0.0, 0.0]
    );
ZshB = [ 0.0+0.0*im 0.0+0.3*im 0.05+0.0*im 0.0+0.0*im 0.0+0.0*im]; 
GshB = [ 0.0 0.0 0.05 0.0 0.0];
BshB = [ 0.0 0.3 0.0 0.0 0.0];
BusNum = 5;
nLine= nrow(BranchData);

In [10]:
R = BranchData.R; X = BranchData.X; Bsh = BranchData.Bsh; T = BranchData.T; ϕ = BranchData.ϕ;
Y = conj((R + X*im).^-1); G = real(Y); B = imag(Y);
insertcols!(BranchData, 4, Y = round.(Y,digits = 3));
insertcols!(BranchData, 5, G = round.(G,digits = 3));
insertcols!(BranchData, 6, B = round.(B,digits = 3));
select!(BranchData, Not(:R));
select!(BranchData, Not(:X));
BranchData

,From,To,Y,G,B,Gsh,Bsh,T,Tl
,Int64,Int64,Complex…,Float64,Float64,Float64,Float64,Float64,Float64
1,1,2,0.0+3.333im,0.0,3.333,0.0,0.0,1.0,1.0
2,1,3,1.067+6.727im,1.067,6.727,0.0,0.04,1.0,1.0
3,2,4,5.66+30.189im,5.66,30.189,0.0,0.01,1.0,1.0
4,3,4,0.294+3.824im,0.294,3.824,0.0,0.0,1.0,1.0
5,3,5,0.0+3.125im,0.0,3.125,0.0,0.0,0.98,0.95
6,4,5,0.0+2.0im,0.0,2.0,0.0,0.0,1.0,1.0


In [11]:
Y   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Y,BusNum,BusNum));
G   = real(Y); B = imag(Y);
Gsh = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Gsh,BusNum,BusNum));
Bsh = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.Bsh,BusNum,BusNum));

In [12]:
T   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.T,BusNum,BusNum));
ϕ   = Symmetric(sparse(BranchData.From,BranchData.To,BranchData.ϕ,BusNum,BusNum));

In [13]:
BusData = DataFrame(Bus = [ 1, 2, 3, 4, 5], 
    PL = [ 0.0, 0.0, 0.0, 0.90, 0.239], QL = [ 0.0, 0.0, 0.0, 0.400, 0.129], 
    Vmin = [ 1.0, 0.95, 0.95, 0.95, 0.95], Vmax = [ 1.0, 1.05, 1.05, 1.05, 1.05]
    );
PL   = BusData.PL;
QL   = BusData.QL;
Vmin = BusData.Vmin;
Vmax = BusData.Vmax;

In [14]:
GenData = DataFrame(Bus = [ 1, 2, 3, 4, 5], 
    PGmin = [ 0.0, 0.0, 0.10, 0.05, 0.0], PGmax = [ 1.0, 0.0, 0.40, 0.40, 0.0], 
    QGmin = [ 0.0, 0.0, -0.2, -0.2, 0.0], QGmax = [ 0.4, 0.0, 0.30, 0.20, 0.0]
    );
PgL = GenData.PGmin;
PgU = GenData.PGmax;
QgL = GenData.QGmin;
QgU = GenData.QGmax;

In [26]:
model = nothing
model = Model(with_optimizer(Mosek.Optimizer,QUIET=false,MSK_IPAR_INFEAS_REPORT_AUTO=true))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek

In [27]:
@variable(model, Pg[1:BusNum]);
@variable(model, Qg[1:BusNum]);

In [28]:
@variable(model, W[i = 1:2*BusNum,j = 1:2*BusNum]);

In [29]:
for i in 1:BusNum
    set_upper_bound(Pg[i], PgU[i]);
    set_lower_bound(Pg[i], PgL[i]);
    set_upper_bound(Qg[i], QgU[i]);
    set_lower_bound(Qg[i], QgL[i]);
end

for i in findall(x->x==0, GenData.PGmax)
    delete_upper_bound(Pg[i])
    delete_lower_bound(Pg[i])
    delete_upper_bound(Qg[i])
    delete_lower_bound(Qg[i])
    fix(Pg[i],0)    
    fix(Qg[i],0)

end

In [30]:
@objective(model, Min, 0.35*Pg[1] + 0.2*Pg[3] + 0.4*Pg[3]*Pg[3] + 0.3*Pg[4] + 0.5*Pg[4]*Pg[4])

0.4 Pg[3]² + 0.5 Pg[4]² + 0.35 Pg[1] + 0.2 Pg[3] + 0.3 Pg[4]

In [31]:
@constraint(model,SOC_Con1[i = 1:BusNum ],   0  <= W[i,i] );
@constraint(model,SOC_Con2[j = 1:BusNum  ],  0  <= W[j,j] );
@constraint(model,SOC_Con3[i = 1:BusNum,j = 1:BusNum], [W[i,i],W[j,j], 1*W[i,j], 1*W[i+BusNum,j] ] in RotatedSecondOrderCone());

@constraint(model,SD_Con1[i = 1:BusNum,j = 1:BusNum ],   W[i,j+BusNum]  == -W[i+BusNum,j] );
@constraint(model,SD_Con2[i = 1:BusNum,j = 1:BusNum ],   W[i,j]  == -W[i+BusNum,j+BusNum] );

In [33]:
@constraint(model, V_Con[i = 1:BusNum], Vmin[i]*Vmin[i]<=W[i,i]<=Vmax[i]*Vmax[i] ); 

In [57]:
δL = -15*π/180; δU = 15*π/180;
@constraint(model, δL_Con[i = 1:BusNum, j = 1:BusNum], tan(δL)*(W[i,j])<= W[i+BusNum,j] );  
@constraint(model, δU_Con[i = 1:BusNum, j = 1:BusNum], W[i+BusNum,j] <= tan(δU)*(W[i,j]) ); 

In [39]:
@expression(model, ActivePFij[i = 1:BusNum,j = 1:BusNum], 
    ( G[i,j]*W[i,i] ) - ( G[i,j]*W[i,j] ) + ( B[i,j]*W[i+BusNum,j] ) );

In [41]:
@constraint(model, ActiveBranch[i = 1:BusNum],Pg[i]-PL[i] == 
sum(ActivePFij[i,j] for j in 1:BusNum) );

In [52]:
@expression(model, ReActivePFij[i = 1:BusNum,j = 1:BusNum], 
    ( B[i,j]*W[i,i] ) - ( G[i,j]*W[i+BusNum,j] ) - ( B[i,j]*W[i,j] ) );

In [53]:
@constraint(model, ReActiveBranch[i = 1:BusNum],Qg[i]-QL[i] == 
sum(ReActivePFij[i,j] for j in 1:BusNum) );

In [58]:
optimize!(model)
objective_value(model)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 179             
  Cones                  : 26              
  Scalar variables       : 166             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Optimizer terminated. Time: 0.02    



0.04025006134268415

In [55]:
model

A JuMP Model
Minimization problem with:
Variables: 110
Objective function type: GenericQuadExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 10 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.Interval{Float64}`: 5 constraints
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`MathOptInterface.RotatedSecondOrderCone`: 25 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Mosek
Names registered in the model: ActiveBranch, ActivePF, ActivePFij, Pg, Qg, RI_Con0, RI_Con1, RI_Con4, ReActiveBranch, ReActivePFij, V_Con, W

In [447]:
P_G = value.(Pg)
Q_G = value.(Qg)
BusResults = DataFrame(Pg = P_G, Qg = Q_G)

,Pg,Qg
,Float64,Float64
1,-2.2542e-12,0.0
2,0.0,0.0
3,0.1,-0.2
4,0.05,-0.2
5,0.0,0.0


In [211]:
value(W[(2,2)])

1.1024999791688326

In [309]:
WR_ = value.(W)
#BranchResults = DataFrame( WR = WR_, WI = WI_) 

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Tuple{Int64,Int64}[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10)  …  (1, 2), (2, 4), (1, 3), (3, 4), (8, 5), (9, 5), (6, 2), (7, 4), (6, 3), (8, 4)]
And data, a 22-element Array{Float64,1}:
  1.0                 
  1.1024999952342072  
  0.9025000033927746  
  1.003565847372876   
  1.1024999979592565  
 -0.0                 
 -0.0                 
 -0.0                 
 -0.0                 
 -0.0                 
  1.4099009902435287  
  0.5127273665108645  
  0.17425745723558203 
  1.201983352079659   
  1.341895577110049   
  0.3458072620124425  
 -0.046815567443898407
 -0.04485783240143266 
  0.015245552777653812
 -0.016968511054689335
  0.06569830236159718 
 -0.030254216082034067

In [390]:
value.(W)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Tuple{Int64,Int64}[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10)  …  (1, 2), (2, 4), (1, 3), (3, 4), (8, 5), (9, 5), (6, 2), (7, 4), (6, 3), (8, 4)]
And data, a 22-element Array{Float64,1}:
  1.0                  
  1.076652264502454    
  0.9218586173714863   
  0.937416117296632    
  1.059810734044951    
 -0.0                  
 -0.0                  
 -0.0                  
 -0.0                  
 -0.0                  
  1.3749936535317637   
  0.4589919614560778   
  0.9681565306458665   
  1.0869386315269105   
  1.277889352195797    
  0.07761220005609751  
 -0.08682726039634896  
  0.018936320785019535 
 -0.07289107419460866  
 -0.009976043845504413 
 -0.007962240003277277 
  0.0012648722975092145

In [88]:
value.(WijR[(1,2)])

6.704912101524865e-8

In [89]:
value.(WijI[(1,2)])

1.8568942831942616e-8

In [147]:
objective_value(model)

-0.31450767514967165

In [29]:
value.(Pg)

5-element Array{Float64,1}:
 -1.7334976431192735e-15
  0.0                   
  2.2191967367655776e-14
  2.3218602875855338e-14
  0.0                   

In [278]:
W

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, Tuple{Int64,Int64}[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (3, 5), (4, 5), (1, 2), (2, 4), (1, 3), (3, 4), (8, 5), (9, 5), (6, 2), (7, 4), (6, 3), (8, 4)]
And data, a 17-element Array{VariableRef,1}:
 W[(1, 1)]
 W[(2, 2)]
 W[(3, 3)]
 W[(4, 4)]
 W[(5, 5)]
 W[(3, 5)]
 W[(4, 5)]
 W[(1, 2)]
 W[(2, 4)]
 W[(1, 3)]
 W[(3, 4)]
 W[(8, 5)]
 W[(9, 5)]
 W[(6, 2)]
 W[(7, 4)]
 W[(6, 3)]
 W[(8, 4)]

In [286]:
value.(W[(1,3)])

0.0